In [ ]:
%pip install --user graphdatascience
%pip install --user neo4j
%pip install --user IProgress
%pip install --user tqdm

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
from dotenv import dotenv_values
from urllib.parse import urljoin
from pprint import pprint as pp
import subprocess
import datetime
import json
from pprint import pprint as pp
from graphdatascience import GraphDataScience

In [ ]:
secret_path = '.secret'
neo4j_connections_file = secret_path + '/' + 'aura_connection.txt'
aura_creds = dotenv_values(neo4j_connections_file)

NEO4J_URI = aura_creds['NEO4J_URI']
NEO4J_USERNAME = aura_creds['NEO4J_USERNAME']
NEO4J_PASSWORD = aura_creds['NEO4J_PASSWORD']

print('NEO4J_URI:', NEO4J_URI)
print('NEO4J_USERNAME:', NEO4J_USERNAME)
print('NEO4J_PASSWORD:', NEO4J_PASSWORD)

In [ ]:
print('NEO4J_URI:', NEO4J_URI) 
print('NEO4J_USERNAME:', NEO4J_USERNAME) 
print('NEO4J_PASSWORD:', NEO4J_PASSWORD) 

gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=True
)
gds.set_database('neo4j')

gds.run_cypher('RETURN gds.version() as version')

### Loading data from CSV

In [ ]:
gds.run_cypher('''
LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/leerazo-demos/london_transport/datasets/London_stations.csv' AS row
MERGE (s:Station {latitude:toFloat(row.Latitude), longitude:toFloat(row.Longitude), name:row.Station, zone:row.Zone})
RETURN count(s) as stations
''')

In [ ]:
gds.run_cypher('''
LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/leerazo-demos/london_transport/datasets/London_tube_lines.csv' as row
MATCH (a:Station), (b:Station) WHERE a.name = row.From_Station AND b.name = row.To_Station
CALL apoc.create.relationship(a, toUpper(row.Tube_Line), {}, b)
YIELD rel as rel1
CALL apoc.create.relationship(b, toUpper(row.Tube_Line), {}, a)
YIELD rel as rel2
RETURN count(rel1) + count(rel2) AS relationships;
''')